<a href="https://colab.research.google.com/github/muskaand26/Intern_assignment/blob/main/Python_%26_Automation_Intern_%E2%80%94_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd

df = pd.read_csv("/content/Data - Sheet1.csv")
print(df.columns.tolist())


['name', 'first_name', 'last_name', 'email', 'created_at', 'approval_status', 'has_joined_event', 'amount', 'amount_tax', 'amount_discount', 'currency', 'ticket_name', 'Job Title', 'What is your LinkedIn profile?']


**Step 1: Data Cleaning**

In [9]:
import pandas as pd


df = df.drop_duplicates(subset="email", keep="first")

df["has_joined_event"] = df["has_joined_event"].str.strip().str.lower().map({
    "yes": True,
    "no": False
})

df["linkedin_flagged"] = df["What is your LinkedIn profile?"].isna() | \
                         (df["What is your LinkedIn profile?"].astype(str).str.strip() == "") | \
                         (~df["What is your LinkedIn profile?"].astype(str).str.contains("linkedin.com", case=False))


df["job_title_blank"] = df["Job Title"].isna() | (df["Job Title"].astype(str).str.strip() == "")

df.to_csv("cleaned_output.csv", index=False)

print("Cleaning complete! Output saved as 'cleaned_output.csv'")


Cleaning complete! Output saved as 'cleaned_output.csv'


**Step 2: Auto-Personalized Messaging**

In [10]:
import pandas as pd
import os
import json

df = pd.read_csv("cleaned_output.csv")

os.makedirs("messages_txt", exist_ok=True)
os.makedirs("messages_json", exist_ok=True)

def generate_message(row):
    name = row.get("first_name") or row.get("name", "there")
    name = name.strip().split()[0] if isinstance(name, str) else "there"
    job_title = row.get("Job Title", "").strip()
    has_joined = row.get("has_joined_event", False)
    linkedin = row.get("What is your LinkedIn profile?", "").strip()
    has_linkedin = "linkedin.com" in linkedin.lower()

    if has_joined:
        message = (
            f"Hey {name}, thanks for joining our session! "
            f"As a {job_title.lower() if job_title else 'professional'}, we think you’ll love our upcoming AI workflow tools. "
            "Want early access?"
        )
    else:
        message = (
            f"Hi {name}, sorry we missed you at the last event! "
            f"We’re preparing another session that might better suit your interests"
            f"{' as a ' + job_title.lower() if job_title else ''}."
        )

    if not has_linkedin:
        message += " Also, feel free to share your LinkedIn for better connections!"

    return message

df["message"] = df.apply(generate_message, axis=1)

df[["email", "message"]].to_csv("personalized_messages.csv", index=False)

for _, row in df.iterrows():
    email = row["email"]
    safe_email = email.replace("@", "_at_").replace(".", "_")
    message = row["message"]


    with open(f"messages_txt/{safe_email}.txt", "w") as txt_file:
        txt_file.write(message)

    with open(f"messages_json/{safe_email}.json", "w") as json_file:
        json.dump({"email": email, "message": message}, json_file, indent=2)

print("Personalized messages generated successfully.")


Personalized messages generated successfully.


**Step 3: Automation (DUMMY)**

In [ ]:
import pandas as pd
import smtplib
from email.mime.text import MIMEText

def send_email(to_email, message):
    from_email = "dummy@example.com"
    msg = MIMEText(message)
    msg['Subject'] = 'Personalized Event Message'
    msg['From'] = from_email
    msg['To'] = to_email

    with smtplib.SMTP('localhost', 1025) as server:
        server.sendmail(from_email, [to_email], msg.as_string())
        print(f"Sent message to {to_email}")

def main():
    df = pd.read_csv('personalized_messages.csv')
    for _, row in df.iterrows():
        send_email(row['email'], row['message'])

if __name__ == "__main__":
    main()
